In [1]:
import pandas as pd
# import torch
# from torch import nn
import datetime
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV , KFold, train_test_split
from sklearn.model_selection import KFold
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import wandb
import pickle
from utils import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

df = pd.read_csv('data/train_processed.csv')
target = 'Target'
IDcol = 'Id'

In [2]:
!pip install xgboost
!pip install imblearn

In [3]:
import pandas as pd
# import torch
# from torch import nn
import datetime
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV , KFold, train_test_split
from sklearn.model_selection import KFold
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import wandb
import pickle
from utils import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [4]:
!pip install xgboost
!pip install imblearn
!pip install wandb

In [5]:
import pandas as pd
# import torch
# from torch import nn
import datetime
import numpy as np
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV , KFold, train_test_split
from sklearn.model_selection import KFold
import seaborn as sb
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
import wandb
import pickle
from utils import *
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams

In [6]:
data_df=load_processed_csv(train=True)
data_df

      feature0  feature1  feature2  feature3  feature4  feature5  feature6  \
0     0.302245       0.0       0.2       0.0       1.0       1.0       0.0   
1    -0.224298       0.0       0.3       0.0       1.0       1.0       1.0   
2     0.206510       0.0       0.7       0.0       1.0       1.0       0.0   
3     0.206510       0.0       0.4       0.0       1.0       1.0       1.0   
4     0.206510       0.0       0.4       0.0       1.0       1.0       1.0   
...        ...       ...       ...       ...       ...       ...       ...   
9552 -0.750840       0.0       0.5       0.0       1.0       1.0       0.0   
9553 -0.750840       0.0       0.5       0.0       1.0       1.0       0.0   
9554 -0.750840       0.0       0.5       0.0       1.0       1.0       0.0   
9555 -0.750840       0.0       0.5       0.0       1.0       1.0       0.0   
9556 -0.750840       0.0       0.5       0.0       1.0       1.0       0.0   

      feature7  feature8  feature9  ...  feature63  feature64  

In [7]:
data_df.head()

   feature0  feature1  feature2  feature3  feature4  feature5  feature6  \
0  0.302245       0.0       0.2       0.0       1.0       1.0       0.0   
1 -0.224298       0.0       0.3       0.0       1.0       1.0       1.0   
2  0.206510       0.0       0.7       0.0       1.0       1.0       0.0   
3  0.206510       0.0       0.4       0.0       1.0       1.0       1.0   
4  0.206510       0.0       0.4       0.0       1.0       1.0       1.0   

   feature7  feature8  feature9  ...  feature63  feature64  feature65  \
0       0.0       0.0     0.125  ...          0          0          2   
1       0.0       0.0     0.125  ...          0          0          1   
2       0.0       0.0     0.000  ...          0          0          1   
3       0.0       0.0     0.250  ...          0          0          1   
4       0.0       0.0     0.250  ...          0          0          1   

   feature66  feature67  feature68  feature69  feature70  feature71  label  
0          1          0          

In [8]:
plt.figure(figsize=(50,50))
sb.heatmap(data_df.corr(), xticklabels=data_df.corr().columns, yticklabels=data_df.corr().columns, cmap='RdYlGn', center=0, annot=True)

<AxesSubplot:>

In [9]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params_xgb, dtrain , 700, vals=[(dtrain,'eval'),(dtest, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [10]:
X = data_df.iloc[:, :-1]
Y = data_df["label"]
print("original distribution:",Counter(Y))
#upsamlping 
smo = SMOTE(random_state=42)
X_smo, Y_smo = smo.fit_resample(X, Y)
print("smote distribution:",Counter(Y_smo))
X,Y=X_smo, Y_smo#using smote dataset

In [11]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [12]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, dtrain , 700, vals=[(dtrain,'eval'),(dtest, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [13]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [14]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, dtrain , 700, vals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [15]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [16]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 700, vals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [17]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 700, vals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [18]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [19]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 700, evals=[(dtrain,'eval'),(dtest, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [20]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [21]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 700, evals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [22]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [23]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 200, evals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [24]:
def xgboost_train(X_train,y_train,X_test,y_test,params=None):
    train_data = xgb.DMatrix(X_train, y_train)
    validation_data = xgb.DMatrix(X_test, y_test)
    if params is None:
        params = {
'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':7, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2
}
    clf = xgb.train(params, train_data , 200, evals=[(train_data,'eval'),(validation_data, 'eval')], verbose_eval=True)
    return clf

def k_fold(k,X,Y,shuffle=False,random_state=None,params=None):
    X,Y=np.array(X),np.array(Y)
    kf = KFold(n_splits=k,random_state=random_state, shuffle=shuffle)
    for train_index, test_index in kf.split(X):
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        model = xgboost_train(X_train,y_train,X_test,y_test,params)
        yield (model,test_index)
# X = data_df.iloc[:, :-1]
# Y = data_df["label"]
# k_fold(3,X,Y)

In [25]:
X = data_df.iloc[:, :-1]
Y = data_df["label"]
print("original distribution:",Counter(Y))
#upsamlping 
smo = SMOTE(random_state=42)
X_smo, Y_smo = smo.fit_resample(X, Y)
print("smote distribution:",Counter(Y_smo))
X,Y=X_smo, Y_smo#using smote dataset

In [26]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        y_test_pred=model.predict(np.array(X)[test_index]).argmax(axis=1)
        y_test=np.array(Y)[test_index]

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)

In [27]:
wandb.login(key="5bc70e88e5aac2b0cd097233114e75b47f6888e6")
wandb.init(project="test-project", entity="cz4041",id='xgb_depth_test')
for depth in range(2,30):
    params={ 'boosting_type': 'dart',
'objective':'multi:softmax',
'num_class':9,
'max_depth':depth, #Max_depth: The maximum depth of a tree.
'min_child_weight':20, #Min_child_weight: The minimum sum of instance weight needed in a child. Keeping it high prevents the child from being too specific and thus helps to avoid overfitting.
'gamma':1,#Minimum loss reduction required to make a further partition on a leaf node of the tree. Again, the larger the game, the less likely the model will overfit.
'subsample':0.8,#The ratio of rows that are randomly selected prior to growing trees. Subsample can also be used to avoid overfitting
'colsample_bytree':0.7,
'tree_method':'hist',
'eval_metric':'mlogloss',
'eta':0.04,#The learning rate. Keeping it high makes the model learn fast but increases the chance of overfitting at the same time
'alpha': 1,#L1 regularization term
'verbose': 2}
    wandb.config.update(params)
    macro_F1_metric=[]
    for model,test_index in k_fold(5,X,Y,shuffle=True,random_state=1,params=params):#k_fold(k,X,Y,shuffle=False,random_state=None)

        # X_train, X_test, y_train, y_test = train_test_split(X, Y,shuffle = "True", test_size =0.999)
        # y_test_pred = model.predict(X_test).argmax(axis=1)
        X_test = np.array(X)[test_index]
        y_test = np.array(Y)[test_index]
        X_test_Dmax = xgb.DMatrix(X_test)
        y_test_pred=model.predict(X_test_Dmax)

        score = accuracy_score(y_test, y_test_pred)
        print("Accuracy score is " + str(score))
        report=classification_report(y_test, y_test_pred)
        macro_F1_metric.append(eval(report.split("\n")[8].split()[4]))
        with open('xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]), 'wb') as f:
            pickle.dump(model, f)#store as pickle
        print('model saved to xgb_data/xgb_model_{}.pkl'.format(macro_F1_metric[-1]))
        print(report)

        # Build the plot
#         cm = confusion_matrix(y_test, y_test_pred)
#         plt.figure(figsize=(16,7))
#         sb.set(font_scale=1.4)
#         sb.heatmap(cm, annot=True, annot_kws={'size':18},cmap=plt.cm.Greens, fmt=".0f")

#         plt.xlabel('Predicted label')
#         plt.ylabel('True label')
#         plt.title('Confusion Matrix for Random Forest Model')
#         plt.show()
    wandb.log({"avg macro_F1_metric": sum(macro_F1_metric)/len(macro_F1_metric)})
    print("avg macro_F1_metric:",sum(macro_F1_metric)/len(macro_F1_metric),depth)